### File structure: This file is organized into 4 key sections:
* Section 1: Data structure definitions and inits
* Section 2: Function definitions for all primary functions
* Section 3: Function definitions for various helper functions
* Section 4: _main (invoking the primary function to price a basket), also functions to support sensitivity analysis

### Section 1: This is where we define and initialize data structures, import data from files, and load the various packages necessary for the computations

In [1]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

In [2]:
import numpy as np
from numpy import ndarray
import pandas as pd
import math
from scipy.stats import norm
from scipy.stats import t
import warnings
warnings.simplefilter('ignore')
from statsmodels.stats.correlation_tools import corr_nearest

In [3]:
# defining the data structures that would be required for the calculation

cds_data =  [
            [31.80, 35.85,39.92,45.07,51.86],
            [37.92,  42.81, 49.14,54.96,63.65],
            [41.77, 47.96, 53.30,58.93,65.06],
            [31.21, 36.79, 41.16,45.52,51.97],
            [33.00, 39.36,44.65,49.26,57.61],
            ]
CDS5Y = pd.DataFrame(cds_data)
CDS5Y_transposed = CDS5Y.transpose()

corr_data_pearson = [
            [1.000000, 0.874067, 0.830036, 0.917487, 0.849246],
            [0.874067, 1.000000, 0.814210, 0.878369, 0.828025],
            [0.830036, 0.814210, 1.000000, 0.835791, 0.860827],
            [0.917487, 0.878369, 0.835791, 1.000000, 0.863615],
            [0.849246, 0.828025, 0.860827, 0.863615, 1.000000]
            ]

corr_pearson = pd.DataFrame(corr_data_pearson)
corr_data_kendall = [
            [1.000000, 0.884988, 0.830244, 0.916459, 0.859054],
            [0.884988, 1.000000, 0.823408, 0.885704, 0.847208],
            [0.830244, 0.823408, 1.000000, 0.835718, 0.870892],
            [0.916459, 0.885704, 0.835718, 1.000000, 0.871201],
            [0.859054, 0.847208, 0.870892, 0.871201, 1.000000]
            ]
corr_kendall = pd.DataFrame(corr_data_kendall)


L = 0.6 # recovery rate of 40%
delta_t = 1
DF = [0.998801438, 0.997405061, 0.995215319, 0.991248188, 0.986116836]
deg_of_freedom = 3
no_of_sims = 100000
rand_num_type = 'sobol'

### Section 2: Primary function definitions (8 functions). Primary functions are those that perform significant computations such as simulating a Gaussian Copula. The subsequent section (Section 3) contains the definitions for the helper functions.

In [4]:
# function to perform sensitivity analysis - for stressed spreads, correlations and recovery rates

def calculate_stressed_spreads(is_stressed_spreads, is_stressed_corr, is_stressed_L):
    stressed_spreads = [.10, .20, .30, .40, .50]
    stressed_corr = [-.40, -.30, -.20, -.10, .02, .04, .06]
    stressed_L = [.4, .5, .6, .7, .8, .9]
    result_gaussian = []
    result_t = []   
    
    if (is_stressed_spreads):
        for x in stressed_spreads:
            result_gaussian.append(calculate_spreads('gaussian', 'spread', x, 0, 'default_rand_num_type')) 
            result_t.append(calculate_spreads('t', 'spread', x, 0, 'default_rand_num_type'))
        return result_gaussian, result_t
    
    if (is_stressed_corr):
        for x in stressed_corr:
            result_gaussian.append(calculate_spreads('gaussian', 'corr', x, 0, 'default_rand_num_type')) 
            result_t.append(calculate_spreads('t', 'corr', x, 0, 'default_rand_num_type'))
        return result_gaussian, result_t
    
    if (is_stressed_L):
        for x in stressed_L:
            result_gaussian.append(calculate_spreads('gaussian', 'L', x, 0, 'default_rand_num_type')) 
            result_t.append(calculate_spreads('t', 'L', x, 0, 'default_rand_num_type'))
        return result_gaussian, result_t   

In [5]:
# function to assess convergence as a function of number of sims - for both pseudo-random numbers and sobol sequences

def test_convergence():
    num_sims = [1000, 5000, 10000, 25000, 50000, 75000, 100000]
    result_gaussian_pseudo_random = []
    result_gaussian_sobol = []
    result_t_pseudo_random = []
    result_t_sobol = []
    
    for x in num_sims:
        result_gaussian_pseudo_random.append(calculate_spreads('gaussian', 'nostress', 0, x, 'pseudo_random'))
        result_gaussian_sobol.append(calculate_spreads('gaussian', 'nostress', 0, x, 'sobol'))
        result_t_pseudo_random.append(calculate_spreads('t', 'nostress', 0, x, 'pseudo_random'))
        result_t_sobol.append(calculate_spreads('t', 'nostress', 0, x, 'sobol'))   
                              
    return result_gaussian_pseudo_random, result_gaussian_sobol, result_t_pseudo_random, result_t_sobol 

In [6]:
# spread calculation for k = 1 to 5

def calculate_spreads(copula_type, stress_type, stress_size, num_sims, random_num_type):
    
    spreads = []
    df2 = pd.DataFrame()
    global CDS5Y_transposed, corr_pearson, corr_kendall, L, no_of_sims, rand_num_type
    
    if (num_sims != 0):
        no_of_sims = num_sims
        rand_num_type = random_num_type
        
    if(stress_type == 'spread'):
        CDS5Y = pd.DataFrame(cds_data)
        CDS5Y_transposed = CDS5Y.transpose()
        CDS5Y_transposed = CDS5Y_transposed.apply(lambda x: x * (1 + stress_size))
    elif(stress_type == 'corr' and copula_type == "gaussian"):
        corr_pearson = pd.DataFrame(corr_data_pearson)
        corr_pearson = corr_pearson.apply(lambda x: x * (1 + stress_size))
        arr = np.array(corr_pearson)
        np.fill_diagonal(arr, 1)
        corr_pearson = pd.DataFrame(arr)
    elif(stress_type == 'corr' and copula_type == "t"):
        corr_kendall = pd.DataFrame(corr_data_kendall)
        corr_kendall = corr_kendall.apply(lambda x: x * (1 + stress_size))
        arr = np.array(corr_kendall)
        np.fill_diagonal(arr, 1)
        corr_kendall = pd.DataFrame(arr)
    elif(stress_type == 'L'):
        L = stress_size

    if(copula_type == 'gaussian'):
        df = pd.DataFrame(simulate_gaussian_copula())
    else:
        df = pd.DataFrame(simulate_t_copula())
    df.columns =['BAC', 'CITI', 'GS', 'JPMC', 'MS'] 
              
    for k in range (5):     
        df['def_flag'] = (df.apply(lambda x: generate_def_flag(x.BAC, x.CITI, x.GS, x.JPMC, x.MS, k+1), axis=1))
        df['def_time'] = (df.apply(lambda x: generate_def_time(x.BAC, x.CITI, x.GS, x.JPMC, x.MS, x.def_flag, k+1), axis=1))
        df['def_payout'] = (df.apply(lambda x: generate_def_payout(x.BAC, x.CITI, x.GS, x.JPMC, x.MS, x.def_flag, x.def_time, k+1), axis=1))
        df['prem_payout'] = (df.apply(lambda x: generate_prem_payout(x.def_flag, x.def_time), axis=1))
        spread = (df['def_payout'].sum()/df['prem_payout'].sum())*10000
        spreads.append(spread)
        #df.to_csv(str(k)+'sim_results.csv')
    return(spreads)
  

In [7]:
# sampling from a simulated gaussian copula

def simulate_gaussian_copula():
    # sampling from a simulated gaussian copula
    
    if (rand_num_type == 'sobol'):
        rand_normal = np.array(pd.read_csv ("C:/Users/anirbighosh/Documents/CQF/Final Project/data files/sobol.csv", header = None, nrows = no_of_sims)) 
    else:
        rand_normal = np.array(pd.read_csv ("C:/Users/anirbighosh/Documents/CQF/Final Project/data files/pseudo_random.csv", header = None, nrows = no_of_sims)) 
      
    rand_correlated = np.transpose(np.dot(calculate_cholesky_matrix(corr_pearson), np.transpose(rand_normal)))
    rand_uniform = pd.DataFrame(norm.cdf(rand_correlated))
    final_input = rand_uniform.apply(lambda x: abs(np.log(1-x)))

    # cumulative hazard rates per entity
    lambda_cum = genarate_cum_hazard_rates(CDS5Y_transposed)
    final_lambdas = lambda_cum.transpose()

    # calling  the get_def_time function recursively to estimate default times for all the 5 entities 
    def_time_all = []
    for i in range(5):
        def_time_all.append(get_def_time(final_input[i], final_lambdas[i]))
    result = np.array(def_time_all).transpose()
    
    #final_lambdas.to_csv(str(L) + 'sim_results.csv')
    return result


In [8]:
# sampling from a simulated "t" copula

def simulate_t_copula():
    
    if (rand_num_type == 'sobol'):
        rand_normal = np.array(pd.read_csv ("C:/Users/anirbighosh/Documents/CQF/Final Project/data files/sobol.csv", header = None, nrows = no_of_sims))       
    else:
        rand_normal = np.array(pd.read_csv ("C:/Users/anirbighosh/Documents/CQF/Final Project/data files/pseudo_random.csv", header = None, nrows = no_of_sims)) 
    
    rand_chisq = pd.read_csv ("C:/Users/anirbighosh/Documents/CQF/Final Project/data files/chisq.csv", header = None, nrows = no_of_sims)
    #rand_chisq = pd.DataFrame(np.random.chisquare(deg_of_freedom, size=(len(rand_normal),5)))
    rand_chisq_mod = rand_chisq.apply(lambda x: np.sqrt(x/deg_of_freedom))
    rand_t = np.divide(rand_normal, np.array(rand_chisq_mod))
    rand_correlated = np.transpose(np.dot(calculate_cholesky_matrix(corr_kendall), np.transpose(rand_t)))
    rand_uniform = pd.DataFrame(t.cdf(rand_correlated, deg_of_freedom))
    final_input = rand_uniform.apply(lambda x: abs(np.log(1-x)))

    # cumulative hazard rates per entity
    lambda_cum = genarate_cum_hazard_rates(CDS5Y_transposed)
    final_lambdas = lambda_cum.transpose()

    # calling  the get_def_time function recursively to estimate default times for all the 5 entities 
    def_time_all = []
    for i in range(5):
        def_time_all.append(get_def_time(final_input[i], final_lambdas[i]))
    result = np.array(def_time_all).transpose()

    return result
    #pd.DataFrame(rand_correlated).to_csv('t_sim_results.csv')

In [9]:
# generating the final cumulative lambda matrix necessary for estimating default times

def genarate_cum_hazard_rates(CDS5Y_transposed):
    # generating the fullsurv prob matrix
    surv_probs_all = []
    for i in range(5):
        surv_probs_all.append(calculate_surv_probs(CDS5Y_transposed[i]))
    result = pd.DataFrame(surv_probs_all).transpose()    
    #print(result)

    #generating the hazard rate matrix
    hazard_rates = result.apply(lambda x: -(np.log(x/x.shift(1)))).transpose()
    hazard_rates.drop(hazard_rates.columns[[0]], axis = 1, inplace = True) 
    #print(hazard_rates)

    # generating the cum hazard rate matrix
    cum_hazard_rates = hazard_rates.assign(sum1 = lambda x: (x[1]), sum2 = lambda x: (x[1])+(x[2]), sum3 = lambda x: (x[1])+(x[2])+(x[3]), sum4 = lambda x: (x[1])+(x[2])+(x[3])+(x[4]), sum5 = lambda x: (x[1])+(x[2])+(x[3])+(x[4])+(x[5]) )
    cum_hazard_rates.drop(cum_hazard_rates.columns[[0,1,2,3,4]], axis = 1, inplace = True)

    return cum_hazard_rates

In [10]:
# bootstrapping piece-wise hazard rates (based on Professor Pena's additional notes on bootstrapping)

def calculate_surv_probs(CDS5Y):
    surv_probs = []
    surv_probs.append(1)

    for j in (0,1,2,3,4):

        if j == 0:
            prob_t1 = L/(L + delta_t * CDS5Y[0]/10000)
            surv_probs.append(prob_t1)
            #print(surv_probs)      
        if j == 1:
            #print("inside j = 1")
            first_term =  get_first_term(CDS5Y, surv_probs, j)
            denominator = get_denominator(CDS5Y, surv_probs, j)
            last_term = get_last_term(CDS5Y, surv_probs, j)
            recurring_term = (first_term)
            surv_prob = (recurring_term/denominator) + last_term
            surv_probs.append(surv_prob)
            #print(surv_probs)
        if j == 2:
            #print("inside j = 2")
            first_term =  get_first_term(CDS5Y, surv_probs, j)
            second_term = get_second_term(CDS5Y, surv_probs, j)
            denominator = get_denominator(CDS5Y, surv_probs, j)
            last_term = get_last_term(CDS5Y, surv_probs, j)
            recurring_term = (first_term + second_term)
            surv_prob = recurring_term/denominator + last_term
            surv_probs.append(surv_prob)
            #print(surv_probs)
        if j == 3:
            #print("inside j = 3")
            first_term =  get_first_term(CDS5Y, surv_probs, j)
            second_term = get_second_term(CDS5Y, surv_probs, j)
            third_term =  get_third_term(CDS5Y, surv_probs, j)
            denominator = get_denominator(CDS5Y, surv_probs, j)
            last_term = get_last_term(CDS5Y, surv_probs, j)
            recurring_term = (first_term + second_term + third_term)
            surv_prob = recurring_term/denominator + last_term
            surv_probs.append(surv_prob)
            #print(surv_probs)
        if j == 4:
            #print("inside j = 4")
            first_term =  get_first_term(CDS5Y, surv_probs, j)
            second_term = get_second_term(CDS5Y, surv_probs, j)
            third_term =  get_third_term(CDS5Y, surv_probs, j)
            fourth_term = get_fourth_term(CDS5Y, surv_probs, j)
            denominator = get_denominator(CDS5Y, surv_probs, j)
            last_term = get_last_term(CDS5Y, surv_probs, j)
            recurring_term = (first_term + second_term + third_term + fourth_term)
            surv_prob = recurring_term/denominator + last_term
            surv_probs.append(surv_prob)
            #print(surv_probs)
            
    return surv_probs

In [11]:
# calculating the cholesky matrix

def calculate_cholesky_matrix(corr):

    # decomposing the matrix using cholesky
    if is_pos_def(corr):
        corr_decomp = np.linalg.cholesky(corr) 
        return corr_decomp 
    else:
        corr_near = corr_nearest(corr, threshold=1e-15, n_fact=100)
        corr_decomp = np.linalg.cholesky(corr_near) 
        return corr_decomp

### Section 3: Helper function definitions (12 functions as of now)

In [12]:
# function to generate def_flag

def generate_def_flag(tau1, tau2, tau3, tau4, tau5, k):
    tau_list = []
    tau_list.extend([tau1, tau2, tau3, tau4, tau5])
    total_def=sum(1 for item in tau_list if (item<5))
    if total_def >= k:
        return 1
    else:
        return 0

In [13]:
# function to generate def_time

def generate_def_time (tau1, tau2, tau3, tau4, tau5, def_flag, k):
    tau_list = []
    tau_list.extend([tau1, tau2, tau3, tau4, tau5])
    tau_list_sorted = sorted (tau_list)
    if def_flag == 1:
        return max(tau_list_sorted[k-1], 0.25)
    else:
        return 0

In [14]:
# function to generate def_payout

def generate_def_payout (tau1, tau2, tau3, tau4, tau5, def_flag, def_time, k):
    expected_loss = 0.2
    
    if def_flag == 1:
        return expected_loss*L*get_DF(def_time)
    else:
        return 0

In [15]:
# function to generate prem_payout (this is based on the simplified formula in Dr. Richard Diamond's notes - Pg 52)

def generate_prem_payout (def_flag, def_time):
    expected_loss = [1, 1, 1, 1, 1]
       
    if def_flag == 1:
        return get_DF(def_time)*def_time
    else:
        return np.dot(expected_loss, DF)

In [16]:
# function to generate DF

def get_DF(def_time):
    coeff_a = 0.0093
    coeff_b = 0.147
    coeff_c = 0.124
    int_rate = coeff_a*(def_time**2) + coeff_b*def_time + coeff_c
    DF = 1/((1+int_rate/100)**def_time)
    return DF

In [17]:
# function to test if a correlation matrix is positive definite

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)


In [18]:
# function to calculate default times from simulated uniform random numbers (using accrual method)

def get_def_time (sims, lambdas):
    def_time = np.array([999.0]*len(sims))
    for iterator1, x in enumerate(sims):
        for iterator2, y in enumerate(lambdas):
            if x < y:
                def_time[iterator1] =  iterator2 + 0.5
                break
    return def_time

In [19]:
# function to calculate the first term in the surv prob matrix

def get_first_term(CDS5Y, surv_probs, j):     
    first_term = DF[0] * ((L * surv_probs[0]) - (L + delta_t * CDS5Y[j]/10000)*surv_probs[1])
    return first_term

In [20]:
# function to calculate the second term in the surv prob matrix

def get_second_term(CDS5Y, surv_probs, j):  
    second_term = DF[1] * ((L * surv_probs[1]) - (L + delta_t * CDS5Y[j]/10000)*surv_probs[2])
    return second_term

In [21]:
# function to calculate the third term in the surv prob matrix

def get_third_term(CDS5Y, surv_probs, j):  
    third_term = DF[2] * ((L * surv_probs[2]) - (L + delta_t * CDS5Y[j]/10000)*surv_probs[3])
    return third_term

In [22]:
# function to calculate the fourth term in the surv prob matrix

def get_fourth_term(CDS5Y, surv_probs, j):  
    third_term = DF[3] * ((L * surv_probs[3]) - (L + delta_t * CDS5Y[j]/10000)*surv_probs[4])
    return third_term

In [23]:
# function to calculate the denominator term in the surv prob matrix

def get_denominator(CDS5Y, surv_probs, j):
    denominator = DF[j] * (L + delta_t * CDS5Y[j]/10000)
    return denominator

In [24]:
# function to calculate the last term in the surv prob matrix

def get_last_term(CDS5Y, surv_probs, j):
    last_term = (surv_probs[j] * L) / (L + delta_t * CDS5Y[j]/10000)
    return last_term

### Section 4: This is where the simulations can be run and outputs generated. Code is provided for 3 use cases: a) calculating spreads (k= 1 to 5) under both the gaussian and 't' copulae, b) calculating spreads after shocking spreads, correlations and recovery rates, and c) assessing convergence

In [27]:
# Run the following functions for calculating spreads under both the guassian and "t" copulae

#calculate_spreads('gaussian', 'nostress', 0, 0, 'default_random_num_type')
#calculate_spreads('t', 'nostress', 0, 0, 'default_random_num_type')

In [28]:
# Run the following function for assessing convergence as a function of the number of simulations

#test_convergence()

In [29]:
# Run the following function for calcuating stressed spreads (stressing spreads, correlations and recovery rates)
# set first param - True for stressing spreads, all others to False
# set second param - True for stressing corr, all others to False
# set thirs param - True for stressing recovery rates, all others to False

# calculate_stressed_spreads(False, True, False)

In [29]:
# functions for random number generation. Note that the random numbers are stored and read from a file - 
# this was done to facilitate debugging. This was also done since I was not able to find a robust sobol generator in Python, 
# hence the data was generated using R and stored in a file. If the random numbers need to generated run-time
# (except sobol), the random number generation codes in "simulate" functions above needs to be uncommented.


#rand_chisq = pd.DataFrame(np.random.chisquare(2, size=(100000,5)))
#rand_chisq.to_csv('chisq.csv')
#rand_normal = pd.DataFrame(np.random.normal(0, 1, size=(100000,5)))
#rand_normal.to_csv('pseudo_random.csv')